# Four point functions in the Potts and $O(n)$ models

This notebook reproduces some known results about Potts and $O(n)$ four-point functions, using the new Julia code.

In [1]:
setprecision(BigFloat, 64)
Pkg.activate("..")
using BootstrapVirasoro,
    Plots

  Activating project at `~/Documents/Recherche/projet_these/code/BootstrapVirasoro`
[ Info: Precompiling BootstrapVirasoro [ca6c609c-7c6b-4ea6-be06-ee6e3ced7d6c] (cache misses: include_dependency fsize change (2), dep missing source (2), mismatched flags (14))
ERROR: Method overwriting is not permitted during Module precompilation. Use `__precompile__(false)` to opt-out of precompilation.
┌ Info: Skipping precompilation due to precompilable error. Importing BootstrapVirasoro [ca6c609c-7c6b-4ea6-be06-ee6e3ced7d6c].
└   exception = Error when precompiling module, potentially caused by a __precompile__(false) declaration in the module.


## Spectra

We create functions to generate the spectra of the $O(n)$ and Potts CFTs, with a cutoff:

\begin{align}
\mathcal{S}^{O(n)} &= \left\{V^d_{\langle 1,s\rangle}\right\}_{s\in 2\mathbb{N}+1} \bigcup \left\{V_{(r,s)}\right\}_{\substack{r\in \frac12\mathbb{N}^*\\ s\in\frac{1}{r}\mathbb{Z}}}  \ ,
 \\
 \mathcal{S}^{PSU(n)} &= \left\{V^d_{\langle 1,s\rangle}\right\}_{s\in\mathbb{N}^*} \bigcup \left\{V_{(r,s)}\right\}_{\substack{r\in \mathbb{N}^*\\ s\in\frac{1}{r}\mathbb{Z}}}  \ ,
 \\
 \mathcal{S}^\text{Potts} &= \left\{V^d_{\langle 1,s\rangle}\right\}_{s\in\mathbb{N}^*} \bigcup \left\{V_{(r,s)}\right\}_{\substack{r\in \mathbb{N}+2\\ s\in\frac{1}{r}\mathbb{Z}}} \bigcup  \left\{ V_{P_{(0,s)}}\right\}_{s\in \mathbb{N}+\frac12}\ .
\end{align}


In [2]:
c = CentralCharge(β=1/(big"0.8" + big"0.1"*im))
Δmax = 10.

indices = ((1//2, 0), (1//2, 0), (1, 0), (1, 0))

fields = [Field(c, r=r, s=s) for (r, s) in indices]
co = Correlation(fields..., Δmax=Δmax)

LoopFields = Dict(
    :On => vcat(
        [Field(c, r=r, s=s) for r in 1//2:1//2:50 for s in -1+1//r:1//r:1 if r*s%1 == 0],
        [Field(c, r=1, s=s, diagonal=true) for s in 1:2:50]
    ),
    :PSUn => vcat(
        [Field(c, r=r, s=s) for r in 1:50 for s in -1+1//r:1//r:1],
        [Field(c, r=1, s=s, diagonal=true) for s in 1:50]
    ),
    :Potts => vcat(
        [Field(c, r=r, s=s) for r in 2:50 for s in -1+1//r:1//r:1],
        [Field(c, r=0, s=s, diagonal=true) for s in 1//2:1:3//2],
        [Field(c, r=1, s=s, diagonal=true) for s in 1:50]
    )
)

LoopSpectra = Dict(
    model => Spectrum(LoopFields[model], Δmax, interchiral=true)
    for model in (:On, :PSUn, :Potts)
)

SOn = LoopSpectra[:On];
@show SOn;

SOn = Degenerate:
<1, 1>
<1, 3>
<1, 5>
<1, 7>
Non-diagonal:
(1, 0), (1, 1)
(2, -1//2), (2, 0), (2, 1//2), (2, 1)
(3, -2//3), (3, -1//3), (3, 0), (3, 1//3), (3, 2//3), (3, 1)


## Bootstrap equations

We numerically solve

\begin{align}
\sum_{V \in \mathcal{S}^{(s)}} D^{(s)}_V \mathcal I^{(s)}_V (x) = \sum_{V \in \mathcal{S}^{(t)}} D^{(t)}_V \mathcal I^{(t)}_V (x) = \sum_{V \in \mathcal{S}^{(u)}} D^{(u)}_V \mathcal I^{(u)}_V (x),
\end{align}

for some channel spectra $\mathcal{S}^{(s)}, \mathcal{S}^{(t)}, \mathcal{S}^{(u)}$, where $\mathcal I$ are interchiral conformal blocks, and the $D$'s are four-point structure constants.

We solve this system as
\begin{equation}
\underbrace{
\begin{bmatrix}
[\mathcal I^{(s)}_{V_j}(x_i)]_{ij} & [-\mathcal I^{(t)}_{V_j}(x_i)]_{ij} & [0] \\
[\mathcal I^{(s)}_{V_j}(x_i)]_{ij} & [0] & [-\mathcal I^{(u)}_{V_j}(x_i)]_{ij}
\end{bmatrix}}_A
\begin{bmatrix}
[D^{(s)}_{V_j}]_j \\
[D^{(t)}_{V_j}]_j \\
[D^{(u)}_{V_j}]_j
\end{bmatrix} = 
\begin{bmatrix}
\sum_{V_j \in \text{ known}} D^{(t)}_{V_j} \mathcal{I}^{(t)}_{V_j}(x_i) - \sum_{V_j \in \text{ known}} D^{(s)}_{V_j} \mathcal{I}^{(s)}_{V_j}(x_i)\\
\sum_{V_j \in \text{ known}} D^{(u)}_{V_j} \mathcal{I}^{(u)}_{V_j}(x_i) - \sum_{V_j \in \text{ known}} D^{(s)}_{V_j} \mathcal{I}^{(s)}_{V_j}(x_i)
\end{bmatrix}
\end{equation}
where the $x_i$ take more values than there are unknowns, i.e. $A$ is a tall rectangular matrix. 

To check numerical convergence, we solve two subsystems and compare the solutions. If the solutions are close, we know the computation has converged.

In [3]:
Schan_On = BootstrapVirasoro.ChannelSpectra(
    co, LoopSpectra[:On],
    signature = Dict(:s => 0, :t => 0, :u => 0)
)
csts = StructureConstants(Schan_On)

In [5]:
import BootstrapVirasoro: evaluate_blocks, random_points
S, consts = Schan_On, csts
system = BootstrapSystem(S, consts);
BootstrapVirasoro.solve!(system);

In [ ]:
@show system.consts;

system.consts = Channel s
Channel u
Channel t



Channel s
Channel u
Channel t
